In [2]:
import pandas as pd

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [34]:
import random
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from pycaret import regression


In [44]:
train = pd.read_csv('./train.csv')
train = train.groupby('SMILES').mean().reset_index()
train['Molecular_Weight'] = [Descriptors.MolWt(Chem.MolFromSmiles(i)) for i in train['SMILES']]
train['Numbond'] = [Chem.MolFromSmiles(i).GetNumBonds() for i in train['SMILES']]
train['HeavyAtoms'] = [Chem.MolFromSmiles(i).GetNumHeavyAtoms() for i in train['SMILES']]
train['AlogP'] = [Descriptors.MolLogP(Chem.MolFromSmiles(i)) for i in train['SMILES']]


droping = []
tp = pd.DataFrame(train['SMILES'].str.contains('e'))
droping.extend(list(tp[tp['SMILES'] == True].index))
tp = pd.DataFrame(train['SMILES'].str.contains('I'))
droping.extend(list(tp[tp['SMILES'] == True].index))
tp = pd.DataFrame(train['SMILES'].str.contains('6'))
droping.extend(list(tp[tp['SMILES'] == True].index))
droping
train = train.drop(droping)

PandasTools.AddMoleculeColumnToFrame(train,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test,'SMILES','Molecule')
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=64)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar

train["FPs"] = train.Molecule.apply(mol2fp)
test["FPs"] = test.Molecule.apply(mol2fp)


mlm_train = train.drop(['SMILES','HLM'],axis=1)
hlm_train = train.drop(['SMILES','MLM'],axis=1)
train = train.reset_index(drop=True)

In [45]:
mlm_train = pd.concat([mlm_train,pd.DataFrame([list(i) for i in train.FPs])],axis=1)
mlm_train = mlm_train.dropna()
mlm_train = mlm_train.reset_index(drop=True)
mlm_train = mlm_train.drop(['FPs','Molecule'],axis=1)

In [46]:
mlm_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3441 entries, 0 to 3440
Data columns (total 74 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   MLM                         3441 non-null   float64
 1   AlogP                       3441 non-null   float64
 2   Molecular_Weight            3441 non-null   float64
 3   Num_H_Acceptors             3441 non-null   float64
 4   Num_H_Donors                3441 non-null   float64
 5   Num_RotatableBonds          3441 non-null   float64
 6   LogD                        3441 non-null   float64
 7   Molecular_PolarSurfaceArea  3441 non-null   float64
 8   Numbond                     3441 non-null   float64
 9   HeavyAtoms                  3441 non-null   float64
 10  0                           3441 non-null   float64
 11  1                           3441 non-null   float64
 12  2                           3441 non-null   float64
 13  3                           3441 

In [47]:
regression.setup(data=mlm_train,target='MLM',
                 fold_strategy='kfold',
                 fold=5,
                 session_id=42)

best = regression.compare_models(sort='RMSE',n_select=5)

,Description,Value
0,Session id,42
1,Target,MLM
2,Target type,Regression
3,Original data shape,"(3441, 74)"
4,Transformed data shape,"(3441, 74)"
5,Transformed train set shape,"(2408, 74)"
6,Transformed test set shape,"(1033, 74)"
7,Numeric features,73
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,27.1662,1039.7976,32.2382,0.1686,1.6727,64.8599,4.7660
et,Extra Trees Regressor,27.8412,1053.0602,32.4443,0.1580,1.7164,71.7508,5.4740
rf,Random Forest Regressor,27.6373,1055.7883,32.4806,0.1561,1.7050,71.8012,5.5620
ada,AdaBoost Regressor,28.7394,1069.2805,32.6927,0.1452,1.7689,79.8596,4.3120
catboost,CatBoost Regressor,27.5309,1076.9069,32.8057,0.1391,1.6780,67.8315,5.7340
omp,Orthogonal Matching Pursuit,28.1045,1077.3458,32.8203,0.1381,1.7115,72.1563,4.5360
llar,Lasso Least Angle Regression,28.4226,1086.4446,32.9561,0.1311,1.7223,74.4500,4.6120
lasso,Lasso Regression,28.4226,1086.4460,32.9562,0.1311,1.7223,74.4499,4.4700
en,Elastic Net,28.7442,1097.9787,33.1285,0.1220,1.7347,76.3384,4.3360
br,Bayesian Ridge,28.6787,1103.2660,33.2080,0.1177,1.7325,76.4285,4.5820


In [ ]:
best_tune = [regression.tune_model(i) for i in best]

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,27.9123,1137.6320,33.7288,0.1057,1.6444,54.1505
1,27.8567,1069.9918,32.7107,0.1411,1.7067,45.1262
2,27.0374,1022.4528,31.9758,0.1987,1.6656,73.7005
3,27.5948,1073.1778,32.7594,0.1481,1.6844,105.3290
4,27.0271,997.4103,31.5818,0.1678,1.7189,64.0861
Mean,27.4857,1060.1329,32.5513,0.1523,1.6840,68.4785
Std,0.3854,48.2232,0.7384,0.0307,0.0270,20.7612


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:58:02
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Results
Estimator,. . . . . . . . . . . . . . . . . .,Extra Trees Regressor


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [11]:
test = pd.read_csv('./test.csv')
test['Molecular_Weight'] = [Descriptors.MolWt(Chem.MolFromSmiles(i)) for i in test['SMILES']]
test['Numbond'] = [Chem.MolFromSmiles(i).GetNumBonds() for i in test['SMILES']]
test['HeavyAtoms'] = [Chem.MolFromSmiles(i).GetNumHeavyAtoms() for i in test['SMILES']]

smiles = test['SMILES'][10]
mol = Chem.MolFromSmiles(smiles)
alogp = Descriptors.MolLogP(mol)
test['AlogP'][10] = alogp

test['AlogP'] = [Descriptors.MolLogP(Chem.MolFromSmiles(i)) for i in test['SMILES']]

test = test.drop(['id','SMILES'],axis=1)

mlm = [i.predict(test) for i in best]
pred_mlm = np.array(mlm).mean(axis=0)

In [12]:
train = pd.read_csv('./train.csv')
train = train.groupby('SMILES').mean().reset_index()
train['Molecular_Weight'] = [Descriptors.MolWt(Chem.MolFromSmiles(i)) for i in train['SMILES']]
train['Numbond'] = [Chem.MolFromSmiles(i).GetNumBonds() for i in train['SMILES']]
train['HeavyAtoms'] = [Chem.MolFromSmiles(i).GetNumHeavyAtoms() for i in train['SMILES']]
train['AlogP'] = [Descriptors.MolLogP(Chem.MolFromSmiles(i)) for i in train['SMILES']]


droping = []
tp = pd.DataFrame(train['SMILES'].str.contains('e'))
droping.extend(list(tp[tp['SMILES'] == True].index))
tp = pd.DataFrame(train['SMILES'].str.contains('I'))
droping.extend(list(tp[tp['SMILES'] == True].index))
tp = pd.DataFrame(train['SMILES'].str.contains('6'))
droping.extend(list(tp[tp['SMILES'] == True].index))
droping
train = train.drop(droping)


hlm_train = train.drop(['SMILES','MLM'],axis=1)

regression.setup(data=mlm_train,target='HLM',
                 fold_strategy='kfold',
                 fold=5,
                 session_id=42)

best = regression.compare_models(sort='RMSE',n_select=5)

,Description,Value
0,Session id,42
1,Target,HLM
2,Target type,Regression
3,Original data shape,"(3456, 10)"
4,Transformed data shape,"(3456, 10)"
5,Transformed train set shape,"(2419, 10)"
6,Transformed test set shape,"(1037, 10)"
7,Numeric features,9
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,28.2708,1101.5625,33.1882,0.1474,1.3549,35.1553,4.8900
br,Bayesian Ridge,28.8309,1111.0873,33.3285,0.1399,1.3753,36.5816,4.8260
ridge,Ridge Regression,28.7700,1111.3469,33.3319,0.1397,1.3719,36.5400,3.7640
lr,Linear Regression,28.7695,1111.3601,33.3321,0.1397,1.3719,36.5401,3.7640
llar,Lasso Least Angle Regression,29.0453,1116.9223,33.4164,0.1354,1.3792,36.8310,5.0960
lasso,Lasso Regression,29.0455,1116.9333,33.4166,0.1354,1.3792,36.8316,3.7620
en,Elastic Net,29.2372,1120.6475,33.4734,0.1326,1.3841,37.3639,3.8020
lar,Least Angle Regression,28.7112,1122.1938,33.4929,0.1313,1.3648,36.2771,4.6000
ada,AdaBoost Regressor,29.3433,1123.0224,33.5110,0.1307,1.3851,36.3511,4.5020
rf,Random Forest Regressor,28.5465,1130.8789,33.6257,0.1248,1.3589,35.3437,4.9000


In [13]:
test = pd.read_csv('./test.csv')
test['Molecular_Weight'] = [Descriptors.MolWt(Chem.MolFromSmiles(i)) for i in test['SMILES']]
test['Numbond'] = [Chem.MolFromSmiles(i).GetNumBonds() for i in test['SMILES']]
test['HeavyAtoms'] = [Chem.MolFromSmiles(i).GetNumHeavyAtoms() for i in test['SMILES']]

smiles = test['SMILES'][10]
mol = Chem.MolFromSmiles(smiles)
alogp = Descriptors.MolLogP(mol)
test['AlogP'][10] = alogp

test['AlogP'] = [Descriptors.MolLogP(Chem.MolFromSmiles(i)) for i in test['SMILES']]

test = test.drop(['id','SMILES'],axis=1)

hlm = [i.predict(test) for i in best]
pred_hlm = np.array(hlm).mean(axis=0)

In [14]:
sub = pd.read_csv('./sample_submission.csv')
sub['MLM'] = pred_mlm
sub['HLM'] = pred_hlm
sub = sub.set_index('id')
sub.to_csv('./submission.csv')